In [1]:
import cv2
import numpy as np
import time

In [8]:
# 显示视频demo
cap = cv2.VideoCapture('../data/vehicle.mp4')
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
fps = cap.get(cv2.CAP_PROP_FPS)
while True:
#     t =time.time()
    ret,frame = cap.read()
    if ret:
        cv2.imshow('ex',frame)
    key = cv2.waitKey(int(1000/fps))
    if key== ord('q') or key == ord('\033'): # esc : 27
        break
#     print(time.time()-t)    
cap.release()
cv2.destroyAllWindows()    

In [64]:
# 给视频加上背景减除

cap = cv2.VideoCapture('../data/vehicle.mp4')
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
fps = cap.get(cv2.CAP_PROP_FPS)
# 创建背景减除器对象
bgs = cv2.createBackgroundSubtractorMOG2() # 3.4.1.14

while True:
    #  t =time.time()
    ret,frame = cap.read()
    if ret:
        fmask = bgs.apply(frame)
        cv2.imshow('ex',fmask)
    key = cv2.waitKey(int(1000/fps))
    if key== ord('q') or key == ord('\033'): # esc : 27
        break
    # print(time.time()-t)    
cap.release()
cv2.destroyAllWindows()

检测关键步骤

1 完成前背景分离(先高斯模糊,再生成前背景的fmask)

2 前背景分离后噪音去除,(观察有什么噪音,利用高斯模糊,二值化,膨胀腐蚀等操作让fmask和车贴合)

3 找轮廓(要求上一步中保证每个目标都有独立的区域,不连通,否则会被认为是同一辆车)

4 画框(可能要根据高宽,位置等过滤掉一些框)

In [85]:
# 形态学识别车辆

cap = cv2.VideoCapture('../data/vehicle.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
# 创建背景减除器对象
bgs = cv2.createBackgroundSubtractorMOG2() # 3.4.1.14
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(27,27))
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))  
kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT,(51,51))
cv2.namedWindow('src',cv2.WINDOW_NORMAL)
cv2.namedWindow('fmask',cv2.WINDOW_NORMAL)
detect_line = 680
detect_line_th = 10
while cap.isOpened():
    
    #  t =time.time()
    ret,frame = cap.read()
    if ret:
        src =frame.copy()
        src[detect_line:detect_line+detect_line_th,:]=[0,255,0] # 画出计数线
        # frame 转成灰度图进行处理
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #  先高斯模糊降噪,再减除背景
        blur = cv2.GaussianBlur(frame,(3,3),100)
        fmask = bgs.apply(blur)
        # 对提取的背景再次降噪及二值化
        fmask = cv2.GaussianBlur(fmask,(3,3),300)
#         fmask = cv2.medianBlur(fmask,71)
        _,fmask =cv2.threshold(fmask,252,255,cv2.THRESH_BINARY)
    
        # 再次过滤细小白点噪音
        fmask = cv2.erode(fmask,kernel2,iterations =5)
        
        # 把主体链接起来
        fmask = cv2.dilate(fmask,kernel1,iterations=3)
        
        # 缩减白色区域,使其和车外围大小相近
        fmask = cv2.erode(fmask,kernel3,iterations =1)
        # src = cv2.bitwise_and(src,np.tile(fmask.reshape(fmask.shape[0],fmask.shape[1],1),(1,1,3)))
        # 查找轮廓
        fmask, contours, hierarchy = cv2.findContours(fmask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # 画出所有轮廓
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            cv2.rectangle(src,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.imshow('fmask',fmask)
        cv2.imshow('src',src)
        '''-------------------------------'''
        # 先减除背景再降噪
        #fmask = bgs.apply(frame)
        #blur = cv2.GaussianBlur(fmask,(5,5),5)
        #cv2.imshow('ex',blur)
        '''-------------------------------'''
        
#         key = cv2.waitKey(int(1000/fps))
        key = cv2.waitKey(1)
    else:
        break
    if key== ord('q') or key == ord('\033'): # esc : 27
        break
    
    # print(time.time()-t)    
cap.release()
cv2.destroyAllWindows()

In [86]:
# 计数逻辑
# 把框的中心点作为计数基准
def center(x,y,w,h):
    cx = int(x+w/2)
    cy = int(y+h/2)
    return cx,cy

In [118]:
# 画一下中心点的轨迹图,可以观察到如果有部分车在计数区域行驶缓慢,或者中心点抖动,将出现重复计数的现象 
# 如果计数范围过大,则容易出现重复计数,范围太小,又容易导致离散的点不会落在范围内导致漏检

cap = cv2.VideoCapture('../data/vehicle.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

# 创建背景减除器对象
bgs = cv2.createBackgroundSubtractorMOG2() # 3.4.1.14
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(27,27))
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))  
kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT,(51,51))
cv2.namedWindow('src',cv2.WINDOW_NORMAL)
cv2.namedWindow('fmask',cv2.WINDOW_NORMAL)
# cv2.namedWindow('cpoint',cv2.WINDOW_NORMAL)
detect_line = 680
detect_line_th = 15
h =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cpoint_img = np.zeros((h,w,3),dtype=np.uint8)
cpoint_img[detect_line:detect_line+detect_line_th,:]=[0,255,0] # 画出计数线
count = 0
while cap.isOpened():
    
    #  t =time.time()
    ret,frame = cap.read()
    if ret:
        src =frame.copy()
        src[detect_line:detect_line+detect_line_th,:]=[0,255,0] # 画出计数线
        
        # frame 转成灰度图进行处理
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #  先高斯模糊降噪,再减除背景
        blur = cv2.GaussianBlur(frame,(3,3),100)
        fmask = bgs.apply(blur)
        # 对提取的背景再次降噪及二值化
        fmask = cv2.GaussianBlur(fmask,(3,3),300)
#         fmask = cv2.medianBlur(fmask,71)
        _,fmask =cv2.threshold(fmask,252,255,cv2.THRESH_BINARY)
    
        # 再次过滤细小白点噪音
        fmask = cv2.erode(fmask,kernel2,iterations =5)
        
        # 把主体链接起来
        fmask = cv2.dilate(fmask,kernel1,iterations=3)
        
        # 缩减白色区域,使其和车外围大小相近
        fmask = cv2.erode(fmask,kernel3,iterations =1)
        # src = cv2.bitwise_and(src,np.tile(fmask.reshape(fmask.shape[0],fmask.shape[1],1),(1,1,3)))
        # 查找轮廓
        fmask, contours, hierarchy = cv2.findContours(fmask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # 画出所有轮廓
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            cv2.rectangle(src,(x,y),(x+w,y+h),(0,0,255),2)
            cpoint = center(x,y,w,h)
            cv2.circle(src,cpoint,5,(255,0,0),-1)
            cv2.circle(cpoint_img,cpoint,1,(255,0,0),-1)
            if detect_line<y<detect_line+detect_line_th:
                count +=1
#                 print(count)
            cv2.putText(src,f'count:{count}',(0,100),cv2.FONT_HERSHEY_SIMPLEX,5,(0,255,255),2)    
        cv2.imshow('fmask',fmask)
        cv2.imshow('src',src)
#         cv2.imshow('cpoint_img',cpoint_img)
        '''-------------------------------'''
        # 先减除背景再降噪
        #fmask = bgs.apply(frame)
        #blur = cv2.GaussianBlur(fmask,(5,5),5)
        #cv2.imshow('ex',blur)
        '''-------------------------------'''
        
        key = cv2.waitKey(int(1000/fps))
#         key = cv2.waitKey(500)
    else:
        break
    if key== ord('q') or key == ord('\033'): # esc : 27
        break
    
    # print(time.time()-t)    
cap.release()
cv2.destroyAllWindows()

![image.png](./cpoint.png)

In [113]:
# 通过关联前后两帧坐标来确定车辆的运动状态
pass